# Project MGS (Medical Graphing Study)

This is the first version of project MGS.

In [47]:
# numerical python library needed for mathematical operations
%pip install numpy 			
import numpy as NumPy
print("numpy version: ", NumPy.__version__)
# this library is useful for processing images
%pip install opencv-python 	
import cv2 as OpenCV
print("OpenCV version: ", OpenCV.__version__)
# we will be storing our data in a dataframe
%pip install pandas 	
import pandas as Pandas
print("Pandas version: ", Pandas.__version__)

import os # useful for file handling operations
# pyTorch is a common library for creating neural networks and AI applications 	
#%pip install torch		
# pyTorch dependency	
#%pip install torchvision 	
# pyTorch dependency	
#%pip install torchaudio

#import torch as PyTorch
#import torchaudio as PyTorchAudio
#import torchvision as PyTorchVision


# This command installs the GPU version of PyTorch
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

#print("PyTorch version: ", PyTorch.__version__)
#print("CUDA Available: ", PyTorch.cuda.is_available())
print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
print("Please check if this CUDA is available unless your device does not support CUDA")
print("Uncommect the command to install GPU if your device does support CUDA")

# The images we receive are in DICOM format, therefore, we use pyDICOM
%pip install pydicom

import pydicom as DICOM
print("DICOM Version: ", DICOM.__version__)




Note: you may need to restart the kernel to use updated packages.
numpy version:  1.26.2
Note: you may need to restart the kernel to use updated packages.
OpenCV version:  4.9.0
Pandas version:  2.1.4
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Please check if this CUDA is available unless your device does not support CUDA
Uncommect the command to install GPU if your device does support CUDA

Note: you may need to restart the kernel to use updated packages.
DICOM Version:  2.4.4


# Citations

## DICOM to Numpy Array 
By Esa Anjum (https://stackoverflow.com/a/74886257)

In [48]:
def dicom_to_numpy(ds):
    DCM_Img = ds
    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)
    
    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = 0

    Window_Center = int(DCM_Img.get(0x00281050).value) #Get window center from tag (0028, 1050)
    
    Window_Width = int(DCM_Img.get(0x00281051).value) #Get window width from tag (0028, 1051)

    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)


    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = NumPy.zeros((rows, cols), NumPy.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities
                
    return New_Img

# Step One: Image Acquisition

We are primarily using the [Cancer Imaging Archive (LIDC-IDRI) Dataset](https://www.cancerimagingarchive.net/collection/lidc-idri/) for our project. This dataset provides the following information that we will be using for the project: 
1. Images
2. Metadata
3. Nodule Counts
4. Patient Diagnoses

## Data Acquisition and Usage
1. Load list of patients -> ```Patients: Dataframe```
2. Load patient diagnoses -> ```Patients: Dataframe```
3. Load images from top -> ```Patients: Dataframe``` (Location of Top Views in ```TopImages```) and ```TopImages: List of NumPy Arrays```
4. Load images from side -> ```Patients: Dataframe``` (Location of Side Views in ```SideImages```) and ```SideImages: List of NumPy Arrays```
5. Load images from front -> ```Patients: Dataframe``` (Location of Front Views in ```FrontImages```) and ```FrontImages: List of NumPy Arrays```
6. Metadata -> ```Patients: Dataframe```

## Image Acquisition Functions

In [49]:
def LoadImage(Location:os.path, Show:bool = False) -> OpenCV.Mat:
	Image = DICOM.dcmread(Location)
	Image = dicom_to_numpy(Image)

	if Show:
		OpenCV.imshow('Loaded Image', Image)
		OpenCV.waitKey(0)

	return OpenCV.Mat(Image)

## Implementation 

In [69]:
# add directories to all files in a 'files' list
directories = []
location = './LIDC-IDRI'
for item in os.listdir(location):
	item_path = os.path.join(location, item)
	if os.path.isdir(item_path):
		directories.append(item_path)

newdirs = []
for dir in directories:
	for item in os.listdir(dir):
		item_path = os.path.join(dir, item)
		if os.path.isdir(item_path):
			newdirs.append(item_path)

filedir = []
for dir in newdirs:
	for item in os.listdir(dir):
		item_path = os.path.join(dir, item)
		if os.path.isdir(item_path):
			filedir.append(item_path)

files = []
for dir in filedir:
	for item in os.listdir(dir):
		item_path = os.path.join(dir, item)
		if not os.path.isdir(item_path):
			_, file_extension = os.path.splitext(item_path)
			#print(file_extension)
			if not file_extension == '.xml':
				files.append(item_path)

# Load the data
# Dataset = 
Images = []

for file in files:
	Images.append(LoadImage(file))




TypeError: int() argument must be a string, a bytes-like object or a real number, not 'MultiValue'